In [3]:
# these two things reimport modules if they change (needed for changing eval_functions)
%reload_ext autoreload
%autoreload 2

In [3]:
import eval_functions
from eval_functions import *
import pandas as pd

# make tables interactive
from itables import init_notebook_mode
import itables.options as opt
init_notebook_mode(all_interactive=True, connected=True)
opt.maxBytes=0

In [1]:
NOODLER_VERSION=""
OLD_NOODLER_VERSION="4318f62-17ffaf6"
if NOODLER_VERSION == "":
  NOODLER_VERSION = OLD_NOODLER_VERSION
NOODLER=f"z3-noodler-{NOODLER_VERSION}"
NOODLER_MODEL=f"z3-noodler-model-{NOODLER_VERSION}"
NOODLER_CHECK_MODEL=f"check-model-{NOODLER_VERSION}"
OLD_NOODLER=f"z3-noodler-{OLD_NOODLER_VERSION}"
OLD_NOODLER_MODEL=f"z3-noodler-model-{OLD_NOODLER_VERSION}"
CVC5="cvc5-1.2.0"
CVC5_MODEL="cvc5-model-1.2.0"
Z3="z3-4.13.0"
Z3_MODEL="z3-model-4.13.0"
Z3STR4="z3str4"
OSTRICH="ostrich-1.4pre"
Z3STR3RE="z3strRE"
Z3TRAU="z3-trau-1.1"
Z3ALPHA="z3-alpha-smtcomp2024"

if NOODLER == "":
  NOODLER = OLD_NOODLER

TOOLS = list(dict.fromkeys([ # small hack so that we get list of unique values (i.e. a set, but also in the given order; see https://stackoverflow.com/questions/1653970/does-python-have-an-ordered-set)
    NOODLER,
    OLD_NOODLER,
"z3-noodler-only-stabilization-4318f62-17ffaf6",
    CVC5,
    Z3,
    NOODLER_MODEL,
    OLD_NOODLER_MODEL,
    CVC5_MODEL,
    Z3_MODEL,
    # NOODLER_CHECK_MODEL,
    # Z3STR4,
    # Z3ALPHA,
    # OSTRICH,
    # Z3STR3RE,
    # Z3TRAU,

    # you can add more tools here directly if needed
]))

VBS = [
    # [Z3, CVC5],
    # [NOODLER, CVC5],
    # [NOODLER, Z3],
    # [NOODLER, Z3, CVC5],
]


bench_selection = (
  # Select one:
    "NORMAL"
    # "INT_CONVS"
    # "QF_S"
    # "QF_SLIA"
    # "QF_SNIA"
    # "ZALIGVINDER"
    # "REGEX"
)

if bench_selection == "NORMAL":
  BENCHES = [
      "sygus_qgen",
      "denghang",
      "automatark",
      "stringfuzz",
      "redos",

      "norn",
      "slog",
      "slent",
      "omark",
      "kepler",
      "woorpje",
      "webapp",
      "kaluza",

      "transducer_plus",
      "leetcode",
      "str_small_rw",
      "pyex",
      "full_str_int",

      "snia",
      ]
elif bench_selection == "INT_CONVS":
  # Only benchmarks with to_int/from_int
  BENCHES = [
      "stringfuzz",
      "str_small_rw",
      "full_str_int",
  ]
elif bench_selection == "QF_S":
  BENCHES = [
      "sygus_qgen",
      "automatark",

      "slog",
      "woorpje",
  ]
elif bench_selection == "QF_SLIA":
  BENCHES = [
      "denghang",
      "stringfuzz",

      "norn",
      "slent",
      "transducer_plus",
      "kepler",
      "woorpje",
      "webapp",
      "kaluza",
      "redos",

      "leetcode",
      "str_small_rw",
      "pyex",
      "full_str_int",
  ]
elif bench_selection == "QF_SNIA":
  BENCHES = [
    "snia"
  ]
elif bench_selection == "ZALIGVINDER":
  BENCHES = [
    "zaligvinder"
  ]
elif bench_selection == "REGEX":
  BENCHES = [
    "regex"
  ]

In [7]:
df_all = load_benches(BENCHES, TOOLS, bench_selection)

# TODO VBS are ugly for now, will fix it
for vbs in VBS:
  name = "+".join(vbs)
  df_all = add_vbs(df_all, vbs, name)
  TOOLS.append(name)
  # tool_names_mapping[name] = " + ".join([tool_names_mapping[tool] for tool in vbs])

### Evaluation

In [ ]:
print(simple_table(df_all, TOOLS, BENCHES, separately=False))
print(simple_table(df_all, TOOLS, BENCHES, separately=True))

In [ ]:
TOOL_FOR_COMPARISON = NOODLER
# TOOL_FOR_COMPARISON = NOODLER_MODEL
# TOOL_FOR_COMPARISON = NOODLER_CHECK_MODEL
# TOOL_FOR_COMPARISON = 

In [ ]:
cactus_plot(df_all, TOOLS, start=int(len(df_all)*0.95), height=2, width=16, put_legend_outside=False, logarithmic_y_axis=True,
                  # tool_names=tool_names_mapping,
                  # file_name_to_save="cactus",
                  # num_of_x_ticks=6,
                )

for tool in TOOLS:
    if tool != TOOL_FOR_COMPARISON:
        print(scatter_plot(df_all, TOOL_FOR_COMPARISON, tool,
                          #  xname=tool_names_mapping[NOODLER], yname=tool_names_mapping[tool],
                          #  file_name_to_save=f"{tool_names_mapping[tool]}_vs_{tool_names_mapping[NOODLER]}", show_legend=False, transparent=True
                           ))


### More detailed evaluation

In [ ]:
# check if noodler does not return different result than other solvers (i.e. wrong sat/unsat)
sanity_check(df_all, TOOL_FOR_COMPARISON, [tool for tool in TOOLS if tool!=TOOL_FOR_COMPARISON])

In [ ]:
# get all formulae where noodler gives different result than sat/unsat/unknown/TO/ERR
get_invalid(df_all, TOOL_FOR_COMPARISON)

In [ ]:
get_errors(df_all, TOOL_FOR_COMPARISON)

In [ ]:
get_timeouts(df_all, TOOL_FOR_COMPARISON)

In [ ]:
get_unknowns(df_all, TOOL_FOR_COMPARISON)

In [ ]:
get_solved(df_all, TOOL_FOR_COMPARISON)

In [ ]:
get_sat(df_all, TOOL_FOR_COMPARISON)

In [ ]:
get_unsat(df_all, TOOL_FOR_COMPARISON)

### TODO: For papers (tables and figures with nicer names or something) 

In [37]:
tool_names_mapping = {
    NOODLER : "Z3-Noodler",
    NOODLER_MODEL : "Z3-Noodlerᴹ",
    CVC5 : "cvc5",
    CVC5_MODEL : "cvc5ᴹ",
    Z3 : "Z3",
    Z3_MODEL : "Z3ᴹ",
    Z3STR4 : "Z3stsr4",
    OSTRICH : "OSTRICH",
    Z3STR3RE : "Z3str3RE",
    Z3TRAU : "Z3-Trau",
    OLD_NOODLER : "Z3-Noodler",
}

tool_latex_mapping = {
    NOODLER : "\\ziiinoodler",
    CVC5 : "\\cvcv",
    Z3 : "\\ziii",
    NOODLER_MODEL : "\\ziiinoodlermodel",
    CVC5_MODEL : "\\cvcvmodel",
    Z3_MODEL : "\\ziiimodel",
    Z3STR4 : "\\ziiistriv",
    OSTRICH : "\\ostrich",
    Z3STR3RE : "\\ziiistriiire",
    Z3TRAU : "\\ziiitrau",
    OLD_NOODLER : "\\ziiinoodlerold",
}

# TODO add table generation for latex